In [209]:
import configparser
from datetime import datetime
import os
import pandas as pd
from pyspark.sql.types import TimestampType, DateType
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession 
from pyspark.sql.functions import udf, col
from pyspark.sql.functions import year, month, dayofmonth, hour, weekofyear, dayofweek, date_format,

SyntaxError: trailing comma not allowed without surrounding parentheses (<ipython-input-209-3d6eff59d855>, line 9)

## Import AWS Key and Secret into OS variables

In [162]:
config = configparser.ConfigParser()
config.read('dl.cfg')
os.environ['AWS_ACCESS_KEY_ID']=config['IAM']['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY']=config['IAM']['AWS_SECRET_ACCESS_KEY']
key = os.getenv("AWS_ACCESS_KEY_ID")
secret = os.getenv("AWS_SECRET_ACCESS_KEY")

# Create Spark Session

In [163]:
spark = SparkSession \
        .builder \
        .appName("Sparkify_Lake") \
        .config("spark.jars.packages", "org.apache.hadoop:hadoop-aws:2.7.0") \
        .config("spark.hadoop.fs.s3a.fast.upload","true")\
        .getOrCreate()

In [164]:
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.access.key", key)
spark.sparkContext._jsc.hadoopConfiguration().set("fs.s3a.secret.key", secret)

# Load Songs and Logs Data

In [165]:
load_songs = "s3a://udacity-dend/song_data/A/A/A/TRAAAAK128F9318786.json"
load_logs =  "s3a://udacity-dend/log_data/2018/11/2018-11-01-events.json"

In [166]:
## Spark reads the song data and prints it's schema
song_data = spark.read.json(load_songs)
song_data.printSchema()

root
 |-- artist_id: string (nullable = true)
 |-- artist_latitude: string (nullable = true)
 |-- artist_location: string (nullable = true)
 |-- artist_longitude: string (nullable = true)
 |-- artist_name: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- num_songs: long (nullable = true)
 |-- song_id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- year: long (nullable = true)



In [167]:
## Spark reads the log data and prints it's schema
log_data = spark.read.json(load_logs)
log_data.printSchema()

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: double (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)



# Process Song and Artist Table

### Extract columns to create songs table 

In [168]:
# song_id, title, artist_id, year, duration
songs_table = song_data[['song_id', 'title', 'artist_id', 'year','duration']]
songs_table.show(3)

+------------------+------+------------------+----+--------+
|           song_id| title|         artist_id|year|duration|
+------------------+------+------------------+----+--------+
|SOBLFFE12AF72AA5BA|Scream|ARJNIUY12298900C91|2009|213.9424|
+------------------+------+------------------+----+--------+



### Extract columns to create artists table

In [169]:
# artist_id, name, location, latitude, longitude
artists_table = song_data[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']]
artists_table.show(3)

+------------------+------------+---------------+---------------+----------------+
|         artist_id| artist_name|artist_location|artist_latitude|artist_longitude|
+------------------+------------+---------------+---------------+----------------+
|ARJNIUY12298900C91|Adelitas Way|               |           null|            null|
+------------------+------------+---------------+---------------+----------------+



### Write songs table to parquet files partitioned by year and artist

In [170]:
songs_table.write.mode("overwrite").partitionBy("year", "artist_id").parquet("songs.parquet")
songs_par = spark.read.parquet('songs.parquet')

In [174]:
songs_par.createOrReplaceTempView("songs_view")
s_view = spark.sql("SELECT * FROM songs_view").show(3)

+------------------+------+--------+----+------------------+
|           song_id| title|duration|year|         artist_id|
+------------------+------+--------+----+------------------+
|SOBLFFE12AF72AA5BA|Scream|213.9424|2009|ARJNIUY12298900C91|
+------------------+------+--------+----+------------------+



### Write artists table to parquet files

In [172]:
artists_table.write.mode("overwrite").parquet("artists.parquet")
artists_par = spark.read.parquet('artists.parquet')

In [175]:
artists_par.createOrReplaceTempView("artists_view")
a_view = spark.sql("SELECT * FROM artists_view").show(3)

+------------------+------------+---------------+---------------+----------------+
|         artist_id| artist_name|artist_location|artist_latitude|artist_longitude|
+------------------+------------+---------------+---------------+----------------+
|ARJNIUY12298900C91|Adelitas Way|               |           null|            null|
+------------------+------------+---------------+---------------+----------------+



# Process Log Data

## Get timestamp from ts column in log_data

In [ ]:
get_timestamp = udf(lambda x: datetime.fromtimestamp(int(int(x) / 1000)), TimestampType())
convert_time = log_data.select('*', get_timestamp(log_data.ts).alias('start_time'))
convert_time.show(1)

In [199]:
# Create temporary sql views
convert_time.createOrReplaceTempView("convert_time_log")
song_data.createOrReplaceTempView("song_data_table")

## Combine data from convert_time_log and  song_data_table to create Songplays table.

In [ ]:
songsplays = spark.sql('''
                        SELECT ct.start_time,
                               ct.userid as user_id,
                               level,
                               s.song_id,
                               s.artist_id,
                               ct.sessionId,
                               ct.location,
                               ct.userAgent as user_agent
                          FROM convert_time_log as ct
                          LEFT JOIN song_data_table as s
                            ON ct.artist = s.artist_name
                         WHERE ct.page = "NextSong"
                      ''').show(5, truncate = False)

### Extract columns for users table 

In [206]:
# user_id, first_name, last_name, gender, level
users = log_data[['userId', 'firstName', 'lastName', 'gender', 'level']]
users.show(3)

+------+---------+--------+------+-----+
|userId|firstName|lastName|gender|level|
+------+---------+--------+------+-----+
|    39|   Walter|    Frye|     M| free|
|     8|   Kaylee| Summers|     F| free|
|     8|   Kaylee| Summers|     F| free|
+------+---------+--------+------+-----+
only showing top 3 rows



### Write users table to parquet files

In [208]:
users.write.mode("overwrite").parquet("users.parquet")
users_par = spark.read.parquet('users.parquet')
users_par.select('*').show(3)

+------+---------+--------+------+-----+
|userId|firstName|lastName|gender|level|
+------+---------+--------+------+-----+
|    39|   Walter|    Frye|     M| free|
|     8|   Kaylee| Summers|     F| free|
|     8|   Kaylee| Summers|     F| free|
+------+---------+--------+------+-----+
only showing top 3 rows



###  Extract columns to create time table

In [213]:
# year, month, dayofmonth, hour, weekofyear, date_format
time_data = log_data.select('ts', get_timestamp(log_data.ts).alias('start_time'))
                    
time = time_data.select(
                        date_format('start_time', 'HH:mm:ss').alias('start_time'),\
                        hour('start_time').alias('hour'),\
                        dayofmonth('start_time').alias('day'),\
                        weekofyear('start_time').alias('week'),\
                        month('start_time').alias('month'),\
                        year('start_time').alias('year'),\
                        dayofweek('start_time').alias('weekday')
                        )
time.show(5)

+----------+----+---+----+-----+----+-------+
|start_time|hour|day|week|month|year|weekday|
+----------+----+---+----+-----+----+-------+
|  15:57:10|  15|  1|  44|   11|2018|      5|
|  16:01:46|  16|  1|  44|   11|2018|      5|
|  16:01:46|  16|  1|  44|   11|2018|      5|
|  16:02:12|  16|  1|  44|   11|2018|      5|
|  16:05:52|  16|  1|  44|   11|2018|      5|
+----------+----+---+----+-----+----+-------+
only showing top 5 rows



### Write users table to parquet files

In [214]:
time.write.mode("overwrite").parquet("time.parquet")
time_par = spark.read.parquet('time.parquet')
time_par.select('*').show(3)

+----------+----+---+----+-----+----+-------+
|start_time|hour|day|week|month|year|weekday|
+----------+----+---+----+-----+----+-------+
|  15:57:10|  15|  1|  44|   11|2018|      5|
|  16:01:46|  16|  1|  44|   11|2018|      5|
|  16:01:46|  16|  1|  44|   11|2018|      5|
+----------+----+---+----+-----+----+-------+
only showing top 3 rows

